In [ ]:
import re
import time
import pandas as pd
import requests

from pathlib import Path

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from io import StringIO

import qgrid

In [ ]:
### Main ########################################################################

url = r'http://wtatennis.com/stats'
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
# Wait for button "SHOW MORE" and "click it" to get all rows, rather than just the first 20

def get_full_stats(driver):
    try:
        klass = "widget-footer__more-button"
        elem = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, klass))).click()
        
    finally:
        print("Button clicked")

get_full_stats(driver)

In [ ]:
# Ingest the stats table.
# CAUTION: before executing this code, need to scroll to the bottom of the page
# in the Selenium/Chromium window to load stats for all players.
# FIXME: this needs to be resolved

def get_table(driver):
    try:
        klass = "stats-list__table"
        table = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, klass))
                )
    finally:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tbl = soup.find("table", {"class": klass})
        dfs = pd.read_html(StringIO(str(tbl)))
        return dfs

dfs = get_table(driver)
df = dfs[0]

In [ ]:
df.columns

In [ ]:
new_cols = [x for x in df.columns if not x.startswith('Unnamed: ')]
new_cols

In [ ]:
dfNew = df[new_cols]
dfNew = dfNew.dropna()
qgrid.show_grid(dfNew)

In [ ]:
'''
<button class="btn widget-footer__more-button stats-list__show-more js-show-more-button" data-text="Show More">Show More</button>

<button class="btn widget-footer__more-button stats-list__show-more js-show-more-button" data-text="Show More">Show More</button>

#main-content > div > div.stats-list__content > button

document.querySelector("#main-content > div > div.stats-list__content > button")
'''